In [1]:
pip install easyocr


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 908.3/908.3 kB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.0/146.0 kB 12.8 MB/s eta 0:00:00


In [2]:
pip install regex

In [3]:
import easyocr
import regex as re

In [4]:
reader = easyocr.Reader(['ch_sim','en'])

Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.0% Complete

In [7]:
result1 = reader.readtext('/content/1.png',detail=0)
print(result1)

['Selva', 'DATA MANAGER', '+123-456-7890', '+123-456-7891', 'WWW.XYZI.com', 'hello@XYZI.com', '123 ABC St Chennai,', 'SelVa', 'TamilNadu 600113', 'digitals']


In [8]:
result2 = reader.readtext('2.png',detail=0)
print(result2)

['Amit Kumar', 'CEO & FOUNDER', '123-456-7569', 'hello@global.com', 'TTT', 'global.com', '123 global', 'Erode;', 'GLOBAL', 'TamilNadu 600115', 'INSURANCE', 'St;']


In [9]:
result3 = reader.readtext('3.png',detail=0)
print(result3)

['KARTHICK', 'General Manager', '123 ABC St , Salem,', 'TamilNadu 6004513', '+123-456-7890', 'hello@Borcelle.com', 'WWW. Borcelle.com', 'BORCELCE', 'AIRLINES']


In [10]:
result4 = reader.readtext('4.png',detail=0)
print(result4)

['REVANTH', 'Marketing Executive', '123 ABC St. HYDRABAD, TamilNadu,', '600001', '+91-456-1234', 'hello@CHRISTMAS.com', 'Family', 'WWW.CHRISTMAS.com', 'Restaurant']


In [11]:
result5 = reader.readtext('5.png',detail=0)
print(result5)

['SANTHOSH', 'Technical Manager', '123 ABC St, Tirupur, TamilNadu;', '641603', '+123-456-1234', 'hello@Sun.com', 'WWWSun.com', 'Sun Electricals']


In [12]:
def phno(img):
  num = []
  for i in img:
    if re.findall(r'^[+]',i):
      num.append(i)
    elif(re.findall(r'^\d{3}-\d{3}-\d{4}$',i)):
      num.append(i)
  return num

In [13]:
def email(img):
  for i in img:
    if(re.findall(r'[\w\.-]+@[\w\.-]+',i)):
      return i

In [15]:
def website(img):
  website = ""
  for i in img:
    if re.match(r'^WWW(?=.*\.com)', i):
      website = i
    elif re.match(r'^\w+\.com$', i):
      website = ('WWW.'+i)
  if len(website) ==0:
    website = "Not Available"
  else:
    return website

In [16]:
def name(img):
  for i in img:
    return i

In [17]:
def designation(img):
  for i in img:
    return img[1]

In [18]:
def address(img):
  for i in img:
    if(re.findall(r'^123+\s[\w\.-]+',i)):
      return i[0:10]

In [19]:
def domain(img):
  for i in img[-1]:
    if len(img[-1])> 5:
      return img[-1]
    else:
      return img[-2]

In [20]:
def district(img):
  for i in img:
    if(re.search(r'^123+\s',i)):
      if len(i[10:20])> 6:
        return i[11:20].replace(",","")
    elif (re.search(r'\bErode\b',i)):
      return i.replace(";","")
  return "Not Available"

In [21]:
def pincode(img):
    pincode = None
    for i in img:
        pincode_match = re.search(r'(\d{6})|\b(\d{3}\s*\d{3})\b', i)
        if pincode_match:
            pincode = pincode_match.group(0).replace(' ', '')
    return pincode

In [22]:
def state(img):
    for i in img:
        match = re.search(r'TamilNadu', i)
        if match:
            return match.group()
    return "Not found"

In [23]:
#create a dictionary with all the data
def data(img):
  data = {}
  data['Name'] = name(img)
  data['Designation'] = designation(img)
  data['Domain'] = domain(img)
  data['Contact'] = phno(img)
  data['E-mail'] = email(img)
  data['Website'] = website(img)
  data['Address'] = address(img)
  data['District'] = district(img)
  data['State'] = state(img)
  data['Pincode'] = pincode(img)
  return data

In [24]:
import pandas as pd

In [25]:
df_1 = pd.DataFrame(data(result1))

df_2 = pd.DataFrame(data(result2))
df_3 = pd.DataFrame(data(result3))
df_4 = pd.DataFrame(data(result4))
df_5 = pd.DataFrame(data(result5))

In [26]:
df = pd.concat([df_1,df_2,df_3,df_4,df_5])

In [27]:
df.to_csv('Extracted_data.csv',index=False)

extract_data=pd.read_csv('Extracted_data.csv')
extract_data

,Name,Designation,Domain,Contact,E-mail,Website,Address,District,State,Pincode
0,Selva,DATA MANAGER,digitals,+123-456-7890,hello@XYZI.com,WWW.XYZI.com,123 ABC St,Chennai,TamilNadu,600113
1,Selva,DATA MANAGER,digitals,+123-456-7891,hello@XYZI.com,WWW.XYZI.com,123 ABC St,Chennai,TamilNadu,600113
2,Amit Kumar,CEO & FOUNDER,INSURANCE,123-456-7569,hello@global.com,WWW.global.com,123 global,Erode,TamilNadu,600115
3,KARTHICK,General Manager,AIRLINES,+123-456-7890,hello@Borcelle.com,WWW. Borcelle.com,123 ABC St,Salem,TamilNadu,600451
4,REVANTH,Marketing Executive,Restaurant,+91-456-1234,hello@CHRISTMAS.com,WWW.CHRISTMAS.com,123 ABC St,HYDRABAD,TamilNadu,600001
5,SANTHOSH,Technical Manager,Sun Electricals,+123-456-1234,hello@Sun.com,WWWSun.com,123 ABC St,Tirupur,TamilNadu,641603


In [28]:
extract_data[1:6]

,Name,Designation,Domain,Contact,E-mail,Website,Address,District,State,Pincode
1,Selva,DATA MANAGER,digitals,+123-456-7891,hello@XYZI.com,WWW.XYZI.com,123 ABC St,Chennai,TamilNadu,600113
2,Amit Kumar,CEO & FOUNDER,INSURANCE,123-456-7569,hello@global.com,WWW.global.com,123 global,Erode,TamilNadu,600115
3,KARTHICK,General Manager,AIRLINES,+123-456-7890,hello@Borcelle.com,WWW. Borcelle.com,123 ABC St,Salem,TamilNadu,600451
4,REVANTH,Marketing Executive,Restaurant,+91-456-1234,hello@CHRISTMAS.com,WWW.CHRISTMAS.com,123 ABC St,HYDRABAD,TamilNadu,600001
5,SANTHOSH,Technical Manager,Sun Electricals,+123-456-1234,hello@Sun.com,WWWSun.com,123 ABC St,Tirupur,TamilNadu,641603


In [29]:
pip install nb2xls

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.3/154.3 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.1/58.1 kB 7.3 MB/s eta 0:00:00
  Created wheel for nb2xls: filename=nb2xls-0.1.6-py3-none-any.whl size=9882 sha256=066f6f8c663dd33291f06df29992dc4c8036aeca88bc49a1c3fab19f7f1061b9
  Stored in directory: /root/.cache/pip/wheels/95/24/3b/9756a25d4f0ddc8b731658465e6a145158bd6efc2f2895d45b
Successfully built nb2xls
